In [1]:
import pandas as pd
from neo4j import GraphDatabase

In [2]:
# setup database connection
driver = GraphDatabase.driver('bolt://localhost:7687', auth=('neo4j', 'fb13'))
sess = driver.session()

In [19]:
def res_to_df(res):
    result_list = [record for record in res]
    return pd.DataFrame(result_list, columns=res.keys()) 

def cypherize_str_list(str_list, brackets=True, quotes=True):
    if type(str_list) is not list:
        str_list = [str_list]

    if quotes:
        out = '", "'.join(str_list)
    else:
        out = ", ".join(str_list)

    if brackets:
        out = '["' + out + '"]'

    return out

In [15]:
# identify people with at least two spouses
q = '''
MATCH (p:Person)-[:spouse]-(x)
WITH p.name as person, count(distinct x) as num_spouses, collect(distinct x.name) as spouses
WHERE num_spouses = 2
RETURN person, spouses, num_spouses ORDER BY num_spouses DESC
'''

In [16]:
promiscuous = res_to_df(sess.run(q))
promiscuous

In [18]:
# identify the subset that has children

In [26]:
people = cypherize_str_list(list(promiscuous.person))
q = f'''
MATCH (p:Person) WHERE p.name in {people}
MATCH (p)-[:children]->(x)
RETURN p.name as parent, x.name as child 

UNION

MATCH (p:Person) WHERE p.name in {people}
MATCH (p)<-[:parents]-(x)
RETURN p.name as parent, x.name as child
'''
promiscuous_parents = res_to_df(sess.run(q))

In [29]:
promiscuous_parents.groupby('parent', as_index=False).agg(set)

,parent,child
0,abigail_adams,{john_quincy_adams}
1,adela_capet,"{robert_i_count_of_flanders, matilda_of_flanders}"
2,adelaide_del_vasto,{roger_ii_of_sicily}
3,adelaide_of_anjou,{constance_of_arles}
4,afonso_v_of_portugal,"{joana_crown_princess_of_portugal, john_ii_of_..."
...,...,...
193,william_i_of_wurttemberg,"{sophie_of_wurttemberg, charles_i_of_wurttemberg}"
194,william_iii_of_the_netherlands,"{prince_willem_of_the_netherlands, wilhelmina_..."
195,woodrow_wilson,"{margaret_woodrow_wilson, eleanor_wilson_mcadoo}"
196,yaroslav_i_the_wise,"{sviatoslav_ii_of_kiev, anne_of_kiev, iziaslav..."
